In [1]:
import pandas as pd

In [2]:
spreadsheetId = pd.read_csv('D:\ITSS_DataScience\spotify-rec\song_id.csv')

In [3]:
spreadsheetId.head()

,id
0,0JXXNGljqupsJaZsgSbMZV
1,51pshtuYkgUQnt5huMPbKL
2,7BqBn9nzAq8spo5e7cZ0dJ
3,161DnLWsx1i3u1JT05lzqU
4,1eUGmzzvahJjOSWgDHuRlv


In [4]:
spreadsheetFeature = pd.read_csv('D:\ITSS_DataScience\spotify-rec\song_features.csv')

In [5]:
spreadsheetFeature.head()

,id,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,0JXXNGljqupsJaZsgSbMZV,0.0267,0.684,0.607,0.000307,0.1910,-8.127,0.1000,81.001,0.4980
1,51pshtuYkgUQnt5huMPbKL,0.0135,0.794,0.766,0.000000,0.0785,-3.894,0.0377,127.007,0.5500
2,7BqBn9nzAq8spo5e7cZ0dJ,0.0134,0.635,0.841,0.000000,0.0622,-5.379,0.0422,109.021,0.4240
3,161DnLWsx1i3u1JT05lzqU,0.5110,0.498,0.590,0.000000,0.1070,-4.721,0.0320,145.867,0.0784
4,1eUGmzzvahJjOSWgDHuRlv,0.8560,0.372,0.239,0.000175,0.1680,-11.314,0.0296,113.451,0.1270
